In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR  
from sklearn.model_selection import ParameterGrid


In [6]:
# Memeriksa ketersediaan GPU
# Perangkat akan diatur ke GPU jika tersedia, atau menggunakan CPU jika GPU tidak tersedia.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [7]:
# Definisi arsitektur CNN
class CNN(nn.Module):
    def __init__(self, kernel_size, pooling_type):
        super(CNN, self).__init__()
        # Lapisan konvolusi pertama: menerima input dengan 1 kanal, menghasilkan 32 fitur, dengan padding yang disesuaikan.
        self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_size, padding=(kernel_size // 2))
        # Lapisan pooling pertama: menggunakan MaxPooling atau AveragePooling sesuai parameter.
        self.pool1 = nn.MaxPool2d(2) if pooling_type == 'max' else nn.AvgPool2d(2)
        # Lapisan konvolusi kedua: menerima 32 fitur, menghasilkan 64 fitur, dengan padding yang disesuaikan.
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=(kernel_size // 2))
        # Lapisan pooling kedua: menggunakan MaxPooling atau AveragePooling sesuai parameter.
        self.pool2 = nn.MaxPool2d(2) if pooling_type == 'max' else nn.AvgPool2d(2)
        # Lapisan fully connected pertama: menghubungkan hasil konvolusi ke 128 unit.
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        # Lapisan fully connected kedua: menghasilkan output untuk 10 kelas.
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Proses data: Konvolusi pertama -> Aktivasi ReLU -> Pooling pertama.
        x = self.pool1(torch.relu(self.conv1(x)))
        # Proses data: Konvolusi kedua -> Aktivasi ReLU -> Pooling kedua.
        x = self.pool2(torch.relu(self.conv2(x)))
        # Mengubah dimensi tensor untuk input ke fully connected layer.
        x = x.view(-1, 64 * 7 * 7)
        # Proses data: Fully connected pertama -> Aktivasi ReLU.
        x = torch.relu(self.fc1(x))
        # Proses data: Fully connected kedua (Output logits).
        x = self.fc2(x)
        return x


In [9]:
# Definisi kelas Early Stopping
class EarlyStopping:
    def __init__(self, patience=5, stop_epochs=[]):
        # Parameter `patience` menentukan jumlah epoch tanpa peningkatan sebelum pelatihan dihentikan.
        self.patience = patience
        self.counter = 0  # Menghitung jumlah epoch tanpa peningkatan.
        self.best_loss = None  # Menyimpan nilai loss terbaik yang tercatat.
        self.early_stop = False  # Flag untuk menentukan apakah pelatihan harus dihentikan.
        self.stop_epochs = set(stop_epochs)  # Daftar epoch tertentu yang ditentukan untuk penghentian awal.

    def __call__(self, val_loss, current_epoch):
        # Jika epoch saat ini ada dalam daftar stop_epochs, pelatihan dihentikan langsung.
        if current_epoch in self.stop_epochs:
            self.early_stop = True
            return

        # Memperbarui nilai loss terbaik jika validasi loss saat ini lebih baik.
        if self.best_loss is None or val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0  # Reset counter jika ada peningkatan.
        else:
            self.counter += 1  # Tambah counter jika tidak ada peningkatan.
            # Menghentikan pelatihan jika jumlah epoch tanpa peningkatan melebihi `patience`.
            if self.counter >= self.patience:
                self.early_stop = True


In [10]:
# Hyperparameter 
param_grid = ParameterGrid({
    'kernel_size': [3, 5, 7],
    'pooling_type': ['max', 'avg'],
    'optimizer_type': ['SGD', 'RMSProp', 'Adam'],
    'epochs': [50, 100, 250, 350]
})


In [11]:
# Persiapan data
# Transformasi data mencakup konversi ke tensor dan normalisasi nilai ke rentang -1 hingga 1.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
# Memuat dataset FashionMNIST untuk data latih dan validasi.
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
val_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
# DataLoader digunakan untuk memuat data dalam batch ukuran 64.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # Data latih dengan shuffle.
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)     # Data validasi tanpa shuffle.

# Fungsi untuk melatih dan mengevaluasi model berdasarkan parameter konfigurasi.
def train_and_evaluate(params):
    # Membuat model CNN berdasarkan parameter kernel_size dan pooling_type.
    model = CNN(kernel_size=params['kernel_size'], pooling_type=params['pooling_type']).to(device)
    criterion = nn.CrossEntropyLoss()  # Fungsi loss untuk klasifikasi multi-kelas.

    # Menentukan jenis optimizer berdasarkan konfigurasi.
    if params['optimizer_type'] == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    else:
        optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Scheduler untuk menurunkan learning rate secara bertahap.
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
    # Menggunakan mekanisme early stopping dengan kesabaran 5 epoch.
    early_stopper = EarlyStopping(patience=5, stop_epochs=[25, 38, 13])

    for epoch in range(params['epochs']):
        model.train()  # Mengaktifkan mode pelatihan untuk model.
        running_loss = 0.0  # Menyimpan total loss selama satu epoch pelatihan.

        for inputs, labels in train_loader:  # Iterasi data latih dalam batch.
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()  # Mengatur ulang gradien sebelum backward pass.
            outputs = model(inputs)  # Melakukan forward pass pada data.
            loss = criterion(outputs, labels)  # Menghitung nilai loss.
            loss.backward()  # Backpropagation untuk menghitung gradien.
            optimizer.step()  # Memperbarui parameter model.
            running_loss += loss.item()  # Menyimpan total loss.

        model.eval()  # Mengaktifkan mode evaluasi untuk model.
        val_loss = 0.0  # Menyimpan total validasi loss.
        with torch.no_grad():  # Menonaktifkan gradien untuk validasi.
            for inputs, labels in val_loader:  # Iterasi data validasi dalam batch.
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)  # Melakukan forward pass pada data validasi.
                loss = criterion(outputs, labels)  # Menghitung validasi loss.
                val_loss += loss.item()

        val_loss /= len(val_loader)  # Menghitung rata-rata validasi loss.
        scheduler.step()  # Memperbarui learning rate berdasarkan scheduler.
        # Menampilkan hasil epoch saat ini.
        print(f"Epoch {epoch + 1}/{params['epochs']}, Loss: {running_loss / len(train_loader):.4f}, Val Loss: {val_loss:.4f}")

        # Memeriksa kondisi early stopping berdasarkan validasi loss.
        early_stopper(val_loss, epoch + 1)
        if early_stopper.early_stop:
            print("Early stopping triggered.")
            break

# Iterasi berbagai kombinasi parameter dari grid untuk eksperimen.
for params in param_grid:
    print(f"Evaluating with params: {params}")
    train_and_evaluate(params)


Evaluating with params: {'epochs': 50, 'kernel_size': 3, 'optimizer_type': 'SGD', 'pooling_type': 'max'}
Epoch 1/50, Loss: 0.5302, Val Loss: 0.3647
Epoch 2/50, Loss: 0.3179, Val Loss: 0.3234
Epoch 3/50, Loss: 0.2684, Val Loss: 0.3037
Epoch 4/50, Loss: 0.2415, Val Loss: 0.2601
Epoch 5/50, Loss: 0.2179, Val Loss: 0.2749
Epoch 6/50, Loss: 0.1968, Val Loss: 0.2511
Epoch 7/50, Loss: 0.1805, Val Loss: 0.2352
Epoch 8/50, Loss: 0.1648, Val Loss: 0.2470
Epoch 9/50, Loss: 0.1508, Val Loss: 0.2451
Epoch 10/50, Loss: 0.1388, Val Loss: 0.2321
Epoch 11/50, Loss: 0.0965, Val Loss: 0.2268
Epoch 12/50, Loss: 0.0895, Val Loss: 0.2298
Epoch 13/50, Loss: 0.0860, Val Loss: 0.2350
Early stopping triggered.
Evaluating with params: {'epochs': 50, 'kernel_size': 3, 'optimizer_type': 'SGD', 'pooling_type': 'avg'}
Epoch 1/50, Loss: 0.6400, Val Loss: 0.4605
Epoch 2/50, Loss: 0.3912, Val Loss: 0.3998
Epoch 3/50, Loss: 0.3384, Val Loss: 0.3661
Epoch 4/50, Loss: 0.3044, Val Loss: 0.3145
Epoch 5/50, Loss: 0.2798, Val